## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Activation
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import EarlyStopping
np.set_printoptions(suppress=True)

data = np.load("/home/hsiehch/30s/2D_spect_img/data_RGB.npy")
labels = np.load('/home/hsiehch/30s/2D_spect_img/labels.npy')

data = data / 255
labels = np_utils.to_categorical(labels, 4)

print(data.shape)
print(labels.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(10151, 59, 151, 3)
(10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [3]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(learning_rate=0):
    model = Sequential() 
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same', data_format="channels_last", input_shape = (data.shape[1], data.shape[2], 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(Dense(160, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(4, activation = "softmax"))
    
    adam = Adam(lr=learning_rate)
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])

    return model

batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]
parameter_grid = dict(batch_size=batch_S, learning_rate=learning_rate)
model = KerasClassifier(build_fn = create_model, epochs=80, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=kf, verbose = 1)
grid_result = grid.fit(data, labels)

print('Finish training!')

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Epoch 1/80
8120/8120 [==============================] - 14s 2ms/step - loss: 1.0648 - acc: 0.5517
Epoch 2/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.9434 - acc: 0.5963
Epoch 3/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.8674 - acc: 0.6369
Epoch 4/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.8149 - acc: 0.6628
Epoch 5/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7814 - acc: 0.6752
Epoch 6/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7499 - acc: 0.6851
Epoch 7/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7250 - acc: 0.6993
Epoch 8/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7180 - acc: 0.6985
Epoch 9/80
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7064 - acc: 0.7060
Epoch 10/80
8120/8120 [==============================] -

8121/8121 [==============================] - 11s 1ms/step - loss: 0.9451 - acc: 0.6028
Epoch 3/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.8738 - acc: 0.6298
Epoch 4/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.8155 - acc: 0.6588
Epoch 5/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7876 - acc: 0.6726
Epoch 6/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7659 - acc: 0.6715
Epoch 7/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7369 - acc: 0.6907
Epoch 8/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7209 - acc: 0.6955
Epoch 9/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7097 - acc: 0.7080
Epoch 10/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6888 - acc: 0.7132
Epoch 11/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6795 - acc: 0.7146
Epoch 12/80
8121/8121 [======

8121/8121 [==============================] - 11s 1ms/step - loss: 0.8407 - acc: 0.6387
Epoch 4/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.8095 - acc: 0.6573
Epoch 5/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7719 - acc: 0.6706
Epoch 6/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7462 - acc: 0.6872
Epoch 7/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7243 - acc: 0.6949
Epoch 8/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7019 - acc: 0.7068
Epoch 9/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6823 - acc: 0.7153
Epoch 10/80
8121/8121 [==============================] - 12s 1ms/step - loss: 0.6756 - acc: 0.7205
Epoch 11/80
8121/8121 [==============================] - 12s 1ms/step - loss: 0.6543 - acc: 0.7307
Epoch 12/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6477 - acc: 0.7332
Epoch 13/80
8121/8121 [=====

8121/8121 [==============================] - 10s 1ms/step - loss: 0.7962 - acc: 0.6745
Epoch 5/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.7665 - acc: 0.6851
Epoch 6/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.7449 - acc: 0.6904
Epoch 7/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.7145 - acc: 0.7048
Epoch 8/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.7035 - acc: 0.7110
Epoch 9/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6806 - acc: 0.7229
Epoch 10/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6794 - acc: 0.7180
Epoch 11/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6522 - acc: 0.7303
Epoch 12/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6387 - acc: 0.7391
Epoch 13/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6197 - acc: 0.7502
Epoch 14/80
8121/8121 [====

8121/8121 [==============================] - 11s 1ms/step - loss: 0.7851 - acc: 0.6704
Epoch 6/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7665 - acc: 0.6773
Epoch 7/80
8121/8121 [==============================] - 12s 1ms/step - loss: 0.7348 - acc: 0.6960
Epoch 8/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7229 - acc: 0.6991
Epoch 9/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7101 - acc: 0.7015
Epoch 10/80
8121/8121 [==============================] - 12s 1ms/step - loss: 0.6928 - acc: 0.7154
Epoch 11/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6720 - acc: 0.7222
Epoch 12/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6668 - acc: 0.7256
Epoch 13/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6492 - acc: 0.7366
Epoch 14/80
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6349 - acc: 0.7413
Epoch 15/80
8121/8121 [===

...

8121/8121 [==============================] - 9s 1ms/step - loss: 0.5592 - acc: 0.7739
Epoch 50/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5496 - acc: 0.7833
Epoch 51/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5445 - acc: 0.7871
Epoch 52/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5325 - acc: 0.7902
Epoch 53/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5281 - acc: 0.7960
Epoch 54/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5245 - acc: 0.7973
Epoch 55/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5157 - acc: 0.7979
Epoch 56/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5268 - acc: 0.7912
Epoch 57/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5077 - acc: 0.7962
Epoch 58/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5020 - acc: 0.7999
Epoch 59/80
8121/8121 [=======

8121/8121 [==============================] - 10s 1ms/step - loss: 0.4476 - acc: 0.8346
Epoch 52/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4349 - acc: 0.8372
Epoch 53/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4289 - acc: 0.8329
Epoch 54/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4324 - acc: 0.8365
Epoch 55/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4201 - acc: 0.8367
Epoch 56/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4081 - acc: 0.8403
Epoch 57/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4152 - acc: 0.8462
Epoch 58/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4013 - acc: 0.8440
Epoch 59/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4062 - acc: 0.8473
Epoch 60/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4130 - acc: 0.8458
Epoch 61/80
8121/8121 

8121/8121 [==============================] - 10s 1ms/step - loss: 0.4997 - acc: 0.8114
Epoch 53/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4762 - acc: 0.8164
Epoch 54/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4772 - acc: 0.8200
Epoch 55/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4780 - acc: 0.8170
Epoch 56/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4719 - acc: 0.8221
Epoch 57/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4732 - acc: 0.8159
Epoch 58/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4794 - acc: 0.8169
Epoch 59/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4732 - acc: 0.8175
Epoch 60/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4591 - acc: 0.8277
Epoch 61/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4499 - acc: 0.8287
Epoch 62/80
8121/8121 [==

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 1781.8min finished


Epoch 1/80
10151/10151 [==============================] - 48s 5ms/step - loss: 1.0233 - acc: 0.5621
Epoch 2/80
10151/10151 [==============================] - 17s 2ms/step - loss: 0.8367 - acc: 0.6564
Epoch 3/80
10151/10151 [==============================] - 17s 2ms/step - loss: 0.7756 - acc: 0.6773
Epoch 4/80
10151/10151 [==============================] - 18s 2ms/step - loss: 0.7317 - acc: 0.6998
Epoch 5/80
10151/10151 [==============================] - 17s 2ms/step - loss: 0.6891 - acc: 0.7166
Epoch 6/80
10151/10151 [==============================] - 18s 2ms/step - loss: 0.6642 - acc: 0.7271
Epoch 7/80
10151/10151 [==============================] - 18s 2ms/step - loss: 0.6406 - acc: 0.7369
Epoch 8/80
10151/10151 [==============================] - 17s 2ms/step - loss: 0.6160 - acc: 0.7505
Epoch 9/80
10151/10151 [==============================] - 17s 2ms/step - loss: 0.6035 - acc: 0.7583
Epoch 10/80
10151/10151 [==============================] - 17s 2ms/step - loss: 0.5827 - acc: 0.7719

In [4]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean:%f (STD:%f) with: %r" % (mean, stdev, param))

Mean:0.784455 (STD:0.008346) with: {'batch_size': 30, 'learning_rate': 5e-05}
Mean:0.791745 (STD:0.011336) with: {'batch_size': 30, 'learning_rate': 0.0001}
Mean:0.785440 (STD:0.009543) with: {'batch_size': 30, 'learning_rate': 0.0005}
Mean:0.790464 (STD:0.002632) with: {'batch_size': 30, 'learning_rate': 0.001}
Mean:0.678554 (STD:0.076984) with: {'batch_size': 30, 'learning_rate': 0.005}
Mean:0.780022 (STD:0.006949) with: {'batch_size': 50, 'learning_rate': 5e-05}
Mean:0.785834 (STD:0.001830) with: {'batch_size': 50, 'learning_rate': 0.0001}
Mean:0.788789 (STD:0.011297) with: {'batch_size': 50, 'learning_rate': 0.0005}
Mean:0.786721 (STD:0.011434) with: {'batch_size': 50, 'learning_rate': 0.001}
Mean:0.631465 (STD:0.192235) with: {'batch_size': 50, 'learning_rate': 0.005}
Mean:0.772535 (STD:0.010683) with: {'batch_size': 70, 'learning_rate': 5e-05}
Mean:0.786918 (STD:0.012012) with: {'batch_size': 70, 'learning_rate': 0.0001}
Mean:0.786031 (STD:0.010908) with: {'batch_size': 70, 'lear

In [5]:
grid_result.best_score_

0.7917446522287119

In [6]:
grid_result.best_params_

{'batch_size': 30, 'learning_rate': 0.0001}

In [8]:
np.set_printoptions(threshold=np.nan)
labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
